# FAQ

This Section contains Frequently Asked Questions from our user community. The information in this Section contains feedback from the CASA development team, but its content is not officially validated and is at the user's discretion. We welcome [feedback](https://casa.nrao.edu/index_contact.shtml).






## Astropy in monolithic CASA

*How do I run astropy (or other external Python packages) in a monolithic CASA environment?*

While the CASA team recommends to install the pip-wheel version of CASA into a customized Python environment (see [modular installation instructions](introduction.ipynb#Modular-Packages)), some use cases require packages like astropy to be installed into the monolithic CASA version.

To add astropy to monolithic CASA, one can use the pip3 that comes with casa. One would need to give the absolute path to pip3 inside the CASA distribution, e.g. /[full path]/casa-6.6.3-22-py3.8..el7/bin/pip3 install astropy. If not, then the OS will use the first pip3 it finds in PATH, which would install astropy in the wrong place and potentially for the wrong python version, etc. Astropy can then be run using the python syntax (see [CASA examples](../examples/community/casa6_demo.ipynb)).

Keep in mind that in this case one has installed astropy into the CASA installation, so it is now available for everyone using that installation. For a shared installation with multiple users, one could install CASA twice into different paths and then modify one of them.

If the user running pip3 does not have permission to write inside the casa folder, pip will likely fall back to a --user installation and put the packages in \$HOME/.local. This will be apparent from the messages pip prints out. If this happens, then starting casa with --user-site should find the packages installed in $HOME/.local. However, this does not allow a good separation between the environments, so is not recommended and is at the user's discretion.



## Running pipeline in non-interactive / batch mode

*How do I run the pipeline in a non-interactive mode, to avoid X-server errors with plotms?*

Production pipeline runs are done as HPC scheduler batch jobs (via Slurm). However, when users wish to run the pipeline in non-interactive / batch mode, for example on a high-performance cluster, they may run into the problem that the CASA plotms needs x11 to make plots (while other pipeline tasks use matplotlib). The error message reads as follows:
```
casa.qtapp: cannot connect to X server localhost:10.0
```

Because of this, it is suggested to run a pipeline in non-interactive mode using a virtual buffer, Xvfb. The typical exe cmds sent to hpc scheduler are something like this:

```
xvfb-run -d ${casabin}/mpicasa ${casain}/casa —nologger —nogui -agg -c casa_script.py
```

Note that Ubuntu machines have an old xvfb-run version, so it would be "xvfb-run -a”.

For running the Pipeline from Singularity (“apptainer”) or Docker instances, you can 1) either "bake" xvfb into the container image (e.g. in Docker files, add the installation of xvfb), or 2) do X11 forwarding to the host (needs to open up ports and increase security risk).  Still, one needs to start things from "xvfb-run”.


## Permission error in Mac OS (!update-casa / pip install)

*How do I get around permission errors on Mac OS that prevent me from running !update-casa or pip install commands in a monolithic environment?*

On Mac OS, permission errors inside the CASA app of the monolithic CASA version might appear, even if the user has read/write permissions in the parent directories. This means that commands like *!update-data* and *!python3.8 -m pip install --upgrade pip* may not work and give the following error:

```
ERROR: Could not install packages due to an OSError: [Errno 1] Operation not permitted:
```

To solve this issue, go to "System Settings" on your Mac, then "Privacy & Security", and "Full Disk Access". The "Terminal" option should be given full disk access. After re-starting the terminal and CASA, the problem should be fixed.

## Mass deploy CASA to macOS clients.

To mass deploy CASA on MacOS clients, particularly using an MDM tool such as Jamf or Kandji, the below procedure can be used. This will suppress the preference/warning message in the Terminal, so that the user does not need to enter a local administrator password.

First, the CASA app itself should be distributed by  copy-from-dmg to /Applications.
Next, an administrator needs to make or add to the users' PPPC profiles, with at minimum the following bits:

Privacy & Security:
```
> App Management (Terminal)
> Automation (CASA to control Terminal; falls under AppleEvents)
```

The kandji reference for this can be found [here](https://support.kandji.io/kb/create-a-privacy-preferences-policy-control-pppc-profile). (jamf admins are assumed to already be familiar)

Additionally, ```~/.casa/data``` must be created for each user, otherwise the launch of CASA fails.